In [ ]:
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://app.apollo.io/#/login")
time.sleep(5)  # Allow page to load

# Perform manual login steps here
print("Please log in manually...")
time.sleep(30)  # Adjust time to give yourself time to log in

# Save cookies after login
with open("apollo_cookies.pkl", "wb") as file:
    pickle.dump(driver.get_cookies(), file)
print("Cookies saved successfully!")

# Close browser after saving cookies
driver.quit()

In [ ]:
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# File to store cookies
COOKIES_FILE = "apollo_cookies.pkl"

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait = WebDriverWait(driver, 10)

# Step 1: Load cookies if available
def load_cookies():
    try:
        with open(COOKIES_FILE, "rb") as file:
            cookies = pickle.load(file)
            for cookie in cookies:
                driver.add_cookie(cookie)
        print("Cookies loaded successfully!")
    except FileNotFoundError:
        print("Cookies file not found. Please log in manually to save cookies first.")

# Step 2: Save cookies after manual login
def save_cookies():
    with open(COOKIES_FILE, "wb") as file:
        pickle.dump(driver.get_cookies(), file)
    print("Cookies saved successfully!")

# Step 3: Navigate to login page and attempt to use cookies
driver.get("https://app.apollo.io/#/login")
time.sleep(5)  # Allow page to load

try:
    load_cookies()
    driver.refresh()
    time.sleep(5)
    if "dashboard" in driver.current_url or "people" in driver.current_url:
        print("Logged in using cookies!")
    else:
        raise Exception("Cookies expired or invalid. Manual login required.")
except Exception as e:
    print(e)
    print("Please log in manually...")
    time.sleep(30)  # Allow manual login
    save_cookies()  # Save cookies after successful login

# Step 4: Navigate to "People" tab
try:
    people_tab = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="side-nav-people"]/div')))  # Adjust XPath
    people_tab.click()
    time.sleep(5)
    print("Navigated to 'People' tab!")
except Exception as e:
    print("Navigation to 'People' tab failed:", e)
    driver.quit()
    exit()

# Step 5: Apply job title filter
try:
    job_title_filter = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="main-app"]/div[2]/div/div[2]/div/div/div/div[2]/div[2]/div[2]/div[1]/div/div/div[2]/div[5]/div[1]')))  # Replace XPath
    job_title_filter.click()
    time.sleep(2)

    job_title_option = driver.find_element(By.XPATH, '//*[@id="main-app"]/div[2]/div/div[2]/div/div/div/div[2]/div[2]/div[2]/div[1]/div/div/div[2]/div[5]/div[2]/div[2]/div[1]/div/div/div/div/div/div/div[1]/div/div/div')  # Replace XPath
    job_title_option.click()
    time.sleep(2)
    print("Job title filter applied!")
except Exception as e:
    print("Job title filter failed:", e)
    driver.quit()
    exit()

# Step 6: Close the browser
driver.quit()